# Imports and Functions

In [1]:
from collections import defaultdict
import json
import os
import glob
from typing import List, Callable, Union

import numpy as np
import pandas as pd
from sklearn.metrics import (average_precision_score, mean_absolute_error, root_mean_squared_error,
                             precision_recall_curve, r2_score, roc_auc_score, mean_absolute_percentage_error, auc)
%load_ext autoreload
%autoreload 2

from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from chemprop import models

In [2]:
def parse_indices(idxs):
    """Parses a string of indices into a list of integers. e.g. '0,1,2-4' -> [0, 1, 2, 3, 4]"""
    if isinstance(idxs, str):
        indices = []
        for idx in idxs.split(","):
            if "-" in idx:
                start, end = map(int, idx.split("-"))
                indices.extend(range(start, end + 1))
            else:
                indices.append(int(idx))
        return indices
    return idxs

def prc_auc(targets: List[int], preds: List[float]) -> float:
    """
    Computes the area under the precision-recall curve.

    :param targets: A list of binary targets.
    :param preds: A list of prediction probabilities.
    :return: The computed prc-auc.
    """
    precision, recall, _ = precision_recall_curve(targets, preds)
    return auc(recall, precision)

def get_metric_func(metric: str):
    r"""
    Gets the metric function corresponding to a given metric name.

    Supports:

    * :code:`roc-auc`: Area under the receiver operating characteristic curve
    * :code:`prc-auc`: Area under the precision recall curve
    * :code:`ap`: Average precision from prediction scores
    * :code:`rmse`: Root mean squared error
    * :code:`mae`: Mean absolute error
    * :code:`r2`: Coefficient of determination R\ :superscript:`2`

    :param metric: Metric name.
    :return: A metric function which takes as arguments a list of targets and a list of predictions and returns.
    """
    if metric == 'roc-auc':
        return roc_auc_score

    if metric == 'prc-auc':
        return prc_auc
    
    if metric == 'ap':
        return average_precision_score

    if metric == 'rmse':
        return root_mean_squared_error
    
    if metric == 'mae':
        return mean_absolute_error

    if metric == 'r2':
        return r2_score
    
    raise ValueError(f'Metric "{metric}" not supported.')
    

In [3]:
def evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=None):
    df = pd.read_csv(data_path)
    with open(splits_path, "rb") as json_file:
        split_idxss = json.load(json_file)
    test_indices = [parse_indices(d["test"]) for d in split_idxss]
    test_df = df.iloc[test_indices[0]]
    target_columns=test_df.keys()[-num_tasks:].tolist() if target_columns is None else target_columns

    df_pred_list = []
    files = glob.glob(os.path.join(result_dir, '**', "test_predictions.csv"), recursive=True)
    assert len(files) == 5, f"There should be 5 files; {len(files)} found"
    for file in files:
        df_pred = pd.read_csv(file)[target_columns]
        df_pred_list.append(df_pred)
    df_pred = pd.concat(df_pred_list).groupby(level=0).mean()

    metric_to_func = {metric: get_metric_func(metric) for metric in metrics}

    results = defaultdict(list)
    for column in target_columns:
        for metric, metric_func in metric_to_func.items():
            preds = df_pred[column].tolist()
            targets = test_df[column].tolist()
            results[metric].append(metric_func(targets, preds))
    results = dict(results)

    results_df = pd.DataFrame(results, index=target_columns)
    return results_df

In [4]:
import pandas as pd
import numpy as np
import os
import glob
import json
from collections import defaultdict

def evaluate_results_err(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=None):
    # Load data and test split
    df = pd.read_csv(data_path)
    with open(splits_path, "rb") as json_file:
        split_idxss = json.load(json_file)
    test_indices = [parse_indices(d["test"]) for d in split_idxss]
    test_df = df.iloc[test_indices[0]]
    
    # Determine target columns
    target_columns = test_df.keys()[-num_tasks:].tolist() if target_columns is None else target_columns

    # Gather prediction files
    files = glob.glob(os.path.join(result_dir, '**', "test_predictions.csv"), recursive=True)
    assert len(files) == 5, f"There should be 5 files; {len(files)} found"

    # Create a dictionary to store results for each file
    results = defaultdict(lambda: defaultdict(list))  # Format: results[file][metric] = [metric values for each target]
    
    # Map metric names to their respective functions
    metric_to_func = {metric: get_metric_func(metric) for metric in metrics}

    # Process each prediction file individually
    for file in files:
        df_pred = pd.read_csv(file)[target_columns]

        # For each target column and metric, calculate metrics for the current file
        for column in target_columns:
            preds = df_pred[column].tolist()
            targets = test_df[column].tolist()
            for metric, metric_func in metric_to_func.items():
                metric_value = metric_func(targets, preds)
                results[file][metric].append(metric_value)  # Store the metric value for this file

    # Create a dictionary to store aggregated results
    aggregated_results = defaultdict(dict)

    # Calculate average and standard deviation for each metric across files
    for metric in metrics:
        metric_values = []

        # Gather all metric values for this metric across files
        for file in files:
            metric_values.append(results[file][metric])

        # Convert to a numpy array for easier mean and std calculations
        metric_values = np.array(metric_values)

        # Store mean and std
        aggregated_results[metric]["mean"] = np.mean(metric_values, axis=0)
        aggregated_results[metric]["std"] = np.std(metric_values, axis=0)

    # Convert results to DataFrame format
    individual_results_df = pd.DataFrame(results)
    aggregated_results_df = pd.DataFrame(aggregated_results)

    return aggregated_results_df

In [5]:
def evaluate_results_1(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=None):
    df = pd.read_csv(data_path)
    with open(splits_path, "rb") as json_file:
        split_idxss = json.load(json_file)
    test_indices = [parse_indices(d["test"]) for d in split_idxss]
    test_df = df.iloc[test_indices[0]]
    target_columns=test_df.keys()[-num_tasks:].tolist() if target_columns is None else target_columns

    df_pred_list = []
    files = glob.glob(os.path.join(result_dir, '**', "test_predictions.csv"), recursive=True)
    # assert len(files) == 5, f"There should be 5 files; {len(files)} found"
    for file in files:
        df_pred = pd.read_csv(file)[target_columns]
        df_pred_list.append(df_pred)
    df_pred = pd.concat(df_pred_list).groupby(level=0).mean()

    metric_to_func = {metric: get_metric_func(metric) for metric in metrics}

    results = defaultdict(list)
    for column in target_columns:
        for metric, metric_func in metric_to_func.items():
            preds = df_pred[column].tolist()
            targets = test_df[column].tolist()
            results[metric].append(metric_func(targets, preds))
    results = dict(results)

    results_df = pd.DataFrame(results, index=target_columns)
    return results_df

In [6]:
def evaluate_results_full(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=None, model_path=None):
    df = pd.read_csv(data_path)
    with open(splits_path, "rb") as json_file:
        split_idxss = json.load(json_file)
    test_indices = [parse_indices(d["test"]) for d in split_idxss]
    test_df = df.iloc[test_indices[0]]
    target_columns=test_df.keys()[-num_tasks:].tolist() if target_columns is None else target_columns

    df_pred_list = []
    files = glob.glob(os.path.join(result_dir, '**', "test_predictions.csv"), recursive=True)
    assert len(files) == 5, f"There should be 5 files; {len(files)} found"
    for file in files:
        df_pred = pd.read_csv(file)[target_columns]
        df_pred_list.append(df_pred)
    df_pred = pd.concat(df_pred_list).groupby(level=0).mean()
    # display(df_pred)
    # display(test_df)
    metric_to_func = {metric: get_metric_func(metric) for metric in metrics}
    
    i = 0
    test_preds = pd.DataFrame()
    for column in target_columns:
        test_preds[f'true_{i}'] = test_df[column]
        test_preds[f'predicted_{i}'] = df_pred[column]
        i=i+1
    # display(test_preds)
    results = defaultdict(list)
    for column in target_columns:
        for metric, metric_func in metric_to_func.items():
            preds = df_pred[column].tolist()
            targets = test_df[column].tolist()
            results[metric].append(metric_func(targets, preds))
    results = dict(results)

    results_df = pd.DataFrame(results, index=target_columns)
    
    if model_path is not None:
        model = models.MPNN.load_from_file(model_path)
        total_params = sum(p.numel() for p in model.parameters())
        results_df["model_size"] = total_params
    
    return results_df, test_preds

# QM 9 Gap

In [7]:
data_path = "/home/akshatz/bond_order_free/qm9/dataset/qm9_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/qm9/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/qm9/qm9_gap/results"
num_tasks = 1
metrics = ["mae","rmse", "r2"]
evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=["gap"])

,mae,rmse,r2
gap,0.003219,0.005877,0.98454


# QM 9 U0

In [8]:
data_path = "/home/akshatz/bond_order_free/qm9/dataset/qm9_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/qm9/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/qm9/qm9_u0/results"
num_tasks = 1
metrics = ["mae", "rmse", "r2"]
evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=["u0_atom"])

,mae,rmse,r2
u0_atom,2.167752,14.132761,0.996425


# QM 9 Multitask

In [9]:
data_path = "/home/akshatz/bond_order_free/qm9/dataset/qm9_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/qm9/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/qm9/qm9_multitask/results"
num_tasks = 12
metrics = ["mae", "rmse", "r2"]
evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics)

,mae,rmse,r2
mu,0.347369,0.607233,0.842723
alpha,0.294276,0.824264,0.989407
homo,0.002494,0.004263,0.962277
lumo,0.002470,0.004171,0.992052
gap,0.003377,0.005998,0.983895
r2,18.115125,36.033951,0.983356
zpve,0.000463,0.001533,0.997844
cv,0.134403,0.330809,0.993252
u0_atom,3.727032,15.641594,0.995621
u298_atom,3.754351,15.748834,0.995639


# Multi Molecule

In [10]:
data_path = "/home/akshatz/bond_order_free/multi_molecule/dataset/mult_mol_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/multi_molecule/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/uv_vis/results"
num_tasks = 1
metrics = ["mae", "rmse", "r2"]
evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics)

,mae,rmse,r2
peakwavs_max,16.859979,31.363678,0.910984


# HIV

In [11]:
data_path = "/home/akshatz/bond_order_free/hiv/dataset/hiv_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/hiv/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/hiv/results"
num_tasks = 1
metrics = ["roc-auc", "ap", "prc-auc"]
results = evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=["HIV_active"])
display(results)

,roc-auc,ap,prc-auc
HIV_active,0.768079,0.336912,0.331843


# PCBA Random

In [12]:
def evaluate_results_with_nan_targets(data_path, splits_path, result_dir, num_tasks, metrics, target_columns=None):
    df = pd.read_csv(data_path)
    with open(splits_path, "rb") as json_file:
        split_idxss = json.load(json_file)
    test_indices = [parse_indices(d["test"]) for d in split_idxss]
    test_df = df.iloc[test_indices[0]]
    target_columns=test_df.keys()[-num_tasks:].tolist() if target_columns is None else target_columns

    df_pred_list = []
    files = glob.glob(os.path.join(result_dir, '**', "test_predictions.csv"), recursive=True)
    assert len(files) == 5, f"There should be 5 files; {len(files)} found"
    for file in files:
        df_pred = pd.read_csv(file)[target_columns]
        df_pred_list.append(df_pred)
    df_pred = pd.concat(df_pred_list).groupby(level=0).mean()

    metric_to_func = {metric: get_metric_func(metric) for metric in metrics}

    results = defaultdict(list)
    for column in target_columns:
        for metric, metric_func in metric_to_func.items():
            preds = df_pred[column].tolist()
            # targets = test_df[column].fillna(0.0).tolist()
            targets = test_df[column].tolist()
            
            pairs = [(target, pred) for target, pred in zip(targets, preds) if not np.isnan(target)]
            targets = [item[0] for item in pairs]
            preds = [item[1] for item in pairs]
            
            results[metric].append(metric_func(targets, preds))
    results = dict(results)

    results_df = pd.DataFrame(results, index=target_columns)
    return results_df

In [13]:
data_path = "/home/akshatz/bond_order_free/pcba_random/dataset/pcba_random_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/pcba_random/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/pcba/pcba_random/results"
num_tasks = 128
metrics = ["prc-auc", "ap", "roc-auc"]
evaluate_results_with_nan_targets(data_path, splits_path, result_dir, num_tasks, metrics).mean()

prc-auc    0.210400
ap         0.215105
roc-auc    0.909773
dtype: float64

# PCBA Random NaN

In [14]:
data_path = "/home/akshatz/bond_order_free/pcba_random_nan/dataset/pcba_random_nan_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/pcba_random_nan/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/pcba/pcba_random_nan/results"
num_tasks = 128
metrics = ["prc-auc", "ap", "roc-auc"]
evaluate_results_with_nan_targets(data_path, splits_path, result_dir, num_tasks, metrics).mean()

prc-auc    0.369269
ap         0.375512
roc-auc    0.905497
dtype: float64

# PCBA Scaffold

In [15]:
data_path = "/home/akshatz/bond_order_free/pcba_scaffold/dataset/pcba_scaffold_data_filtered.csv"  # setting NaN targets to 0
splits_path = "/home/akshatz/bond_order_free/pcba_scaffold/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/pcba/pcba_scaffold/results"
num_tasks = 127
metrics = ["prc-auc", "ap", "roc-auc"]
evaluate_results_with_nan_targets(data_path, splits_path, result_dir, num_tasks, metrics).mean()

prc-auc    0.287534
ap         0.291340
roc-auc    0.889468
dtype: float64

# PCQM4MV2

In [7]:
data_path = "/home/akshatz/bond_order_free/pcqm4mv2/dataset/pcqm4mv2_data_filtered.csv"
splits_path = "/home/akshatz/bond_order_free/pcqm4mv2/dataset/splits_filtered.json"
result_dir = "/home/akshatz/benchmarks_chemeleon/pcqm4mv2/results"
num_tasks = 1
metrics = ["mae", "rmse", "r2"]
evaluate_results(data_path, splits_path, result_dir, num_tasks, metrics)

,mae,rmse,r2
homolumogap,0.092909,0.154394,0.982413


# SAMPL

In [18]:
def evaluate_sampl(test_no, result_dir, metrics):
    targets = None
    df_pred_list = []
    files = glob.glob(os.path.join(result_dir, '**', f"pred_SAMPL{test_no}.csv"), recursive=True)
    assert len(files) == 1, f"There should be 1 file; {len(files)} found"
    for file in files:
        df = pd.read_csv(file)
        
        if targets is None:
            targets = df["logP mean"].tolist() if test_no != 9 else df["new_logPexp_reviewed"].tolist()
            
        df_pred = df["logP"]
        df_pred_list.append(df_pred)
    preds = pd.concat(df_pred_list).groupby(level=0).mean().tolist()
    
    metric_to_func = {metric: get_metric_func(metric) for metric in metrics}

    results = defaultdict(list)
    for metric, metric_func in metric_to_func.items():
        results[metric].append(metric_func(targets, preds))
    results = dict(results)

    results_df = pd.DataFrame(results, index=[f"logP - SAMPL{test_no}"])
    return results_df

In [19]:
metrics = metrics = ["mae", "rmse", "r2"]
evaluate_sampl(6, "/home/akshatz/benchmarks_chemeleon/SAMPL/results_sampl_production", metrics)

,mae,rmse,r2
logP - SAMPL6,0.267726,0.289255,0.811672


In [20]:
metrics = metrics = ["mae", "rmse", "r2"]
evaluate_sampl(7, "/home/akshatz/benchmarks_chemeleon/SAMPL/results_sampl_production", metrics)

,mae,rmse,r2
logP - SAMPL7,0.428742,0.620399,0.127307


In [21]:
metrics = metrics = ["mae", "rmse", "r2"]
evaluate_sampl(9, "/home/akshatz/benchmarks_chemeleon/SAMPL/results_sampl_production", metrics)

,mae,rmse,r2
logP - SAMPL9,0.806533,1.005415,0.798624
